# Intro
In this notebook, we briefly demonstrate how you can use pre-built models from the [open model hub](https://www.objectiv.io/docs/open-model-hub/) in conjunction with our [modeling library](https://objectiv.io/docs/modeling/) to quickly build model stacks to answer common product analytics questions.

This example uses real, unaltered data that was collected from objectiv.io with Objectiv’s Tracker. All models in the open model hub are compatible with datasets that have been validated against the [open analytics taxonomy](https://objectiv.io/docs/taxonomy/).

For an overview of all available models, check out the [open model hub docs](https://www.objectiv.io/docs/open-model-hub/).

In [ ]:
from bach_open_taxonomy import ObjectivFrame

## Instantiate the data object
As a first step, the Objectiv Frame object is instantiated. The Objectiv Frame is an extension to Bach DataFrame to use specifically for data that was collected with Objectiv’s Tracker. Bach is Objectiv’s data modeling library. With Bach, you can use familiar Pandas-like DataFrame operations in your notebook. It uses a SQL abstraction layer that enables models to run on the full dataset, and you can output models to SQL with a single command.

The Objectiv Bach api is heavily inspired by the pandas api. We believe this provides a great, generic interface to handle large amounts of data in a python environment while supporting multiple data stores. For more details on Objectiv Bach, visit the docs.

It loads the data as stored by the Objectiv Tracker, makes a few transformations, and sets the right data types.

This object points to the data on which the models from the open model hub will be applied. The `time_aggregation` parameter determines the standard timeframe that is used with aggregation functions from the model hub. Ie. 'YYYY-MM-DD' means that days are used for the time aggregation. Only data starting at `start_date` is used for all following operations.

In [ ]:
df = ObjectivFrame.from_table(time_aggregation='YYYY-MM-DD', start_date='2021-11-16')

## Using the open model hub
The open model hub contains a growing collection of open-source models that cover a wide range of common analytics use cases. The source is available for all models and you're free to make any changes to them. 

All models are in the `.model_hub` namespace of the Objectiv Frame. You can also use `.mh` as a shorthand. 

Let's run a model to see how it works.

### Unique users
Calculating the unique users is one of the models in the model hub. As it is an aggregation model, it is called with `.model_hub.aggregate.unique_users()`. It uses the *time_aggregation* that is set when the Objectiv Frame was instantiated. With `.to_pandas` or `.head()` we immediately query the data to show the results as a pandas object.

In [ ]:
df.model_hub.aggregate.unique_users().to_pandas()

## Combining models
It is also possible to combine models, if the results allow for that. Here unique sessions and unique users are divided to get the average sessions per user per day.

### Average sessions per user per day

In [ ]:
sessions_per_user = df.mh.agg.unique_sessions() / df.mh.agg.unique_users()
sessions_per_user.to_pandas()

## Using filters for models
The models above show results for all hits in the data. The data can also be filtered. In this example, we calculate new users per day, instead of all users that we calculated above.

In order to do so, we can use the model hub again to create the filter.

### Unique new users

In [ ]:
is_first_session = df.mh.filter.is_first_session()

In [ ]:
df.mh.agg.unique_users(filter=is_first_session).to_pandas()

### Conversion
A filter can also be used for conversions. First has to be defined when a hit is a conversion hit. A conversion is defined by a [location](https://objectiv.io/docs/taxonomy/location-contexts) (on the product) and an event type. The example below defines the clicks to our repository as conversions. A name for the conversion event can optionally be specified.

In [ ]:
df.add_conversion_event(location_stack=df.location_stack.json[{'_type': 'LinkContext', 'id': 'cta-repo-button'}:],
                        event_type='ClickEvent',
                        name='github_clicks')

Using the conversion, we can calculated the number of converted users at once by creating a filter and using that filter in the unique users model.

In [ ]:
df.mh.agg.unique_users(filter=df.mh.f.conversion('github_clicks')).to_pandas()

This concludes the open model hub demo.

We hope you’ve gotten a taste of the power and flexibility of the open model hub to quickly answer common product analytics questions. You can take it a lot further and build highly specific model stacks for in-depth analysis and exploration.

For a complete overview of all available and upcoming models, check out the open model hub docs.

The Bach modeling library can also do a whole lot more. It includes a specific set of operations that are designed for effective modeling on datasets that embrace the open analytics taxonomy, and pandas-compatability means you can tap into the rich ML ecosystem that pandas is well-known for.

There is another notebook in the same folder that demonstrates what you can do with the Bach modeling library [open-taxonomy-how-to.ipynb](open-taxonomy-how-to.ipynb)